In [1]:
%matplotlib inline
from config_calc import *
import grid_tools

from collections import OrderedDict

import cesm_orcas_sci as cesm
import cam
import gv
import metpy.calc as mcalc

import matplotlib.pyplot as plt

### read model data set

In [2]:
case = 'bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004'

tracer_def = cesm.trace_gas_tracers(case)
tracer_3d = [k for k in tracer_def]
tracer_3d = ['CO2','CO2_OCN','CO2_FFF','CO2_LND','O2_OCN']
varlist = ['Z3','Q','PS','PBLH','T','U','V','U10','PSL','OMEGA']+tracer_3d

dsa = cesm.open_casedata(case,'atm','cam.h0',varlist)

### read ORCAS flight data
Find the right day in the model data and subset; store a list of obs datasets 

In [3]:
RF = []
MDL = []
for f in gv.flight_file:
    rf = xr.open_dataset(f)
    RF.append(rf)
    MDL.append(dsa.sel(time=rf.time[0].astype('M8[D]'),method='nearest'))
dss = xr.concat(MDL,dim='time')

### compute pressure from hybrid levels

In [4]:
%%time
dss = cam.pres_hybrid(dss,layer_center=True)
varlist = varlist+['Pm']

#-- convert units of tracer consituents
ds = cesm.convert_dataset(dss,case)
ds = ds.compute()
print ds

<xarray.Dataset>
Dimensions:       (ilev: 57, lat: 192, lev: 56, lon: 288, nbnd: 2, slat: 191, slon: 288, time: 18)
Coordinates:
  * ilev          (ilev) float64 1.651 2.085 2.62 3.276 4.077 5.047 6.217 ...
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 -86.23 -85.29 ...
  * lev           (lev) float64 1.868 2.353 2.948 3.677 4.562 5.632 6.918 ...
  * lon           (lon) float64 0.0 1.25 2.5 3.75 5.0 6.25 7.5 8.75 10.0 ...
  * slat          (slat) float64 -89.53 -88.59 -87.64 -86.7 -85.76 -84.82 ...
  * slon          (slon) float64 -0.625 0.625 1.875 3.125 4.375 5.625 6.875 ...
  * time          (time) datetime64[ns] 2016-01-15 2016-01-18 2016-01-21 ...
Dimensions without coordinates: nbnd
Data variables:
    P0            (time) float64 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 1e+05 ...
    Z3            (time, lev, lat, lon) float32 44901.1 44901.1 44901.1 ...
    ch4vmr        (time) float64 1.846e-06 1.846e-06 1.847e-06 1.847e-06 ...
    co2vmr        (time) float64 0.000403

### interpolate in xy to get a collection of model columns

In [5]:
%%time
model_colums = []
for i,rf in enumerate(RF):
    var = [ds[v].isel(time=i) for v in varlist]
    model_colums.append(cam.interp_columns_esmf(rf.GGLON,rf.GGLAT,ds.lon,ds.lat,*var))
print model_colums[0]

<xarray.Dataset>
Dimensions:  (lev: 56, time: 2723)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-15T12:57:26.000000043 ...
Dimensions without coordinates: lev
Data variables:
    O2_OCN   (time, lev) float64 40.14 40.49 41.32 41.79 41.82 42.3 43.07 ...
    PS       (time) float64 990.8 990.8 990.8 990.8 990.8 990.8 990.8 990.8 ...
    PBLH     (time) float64 687.3 685.6 683.3 680.7 678.2 676.2 674.8 674.3 ...
    CO2      (time, lev) float64 381.3 381.5 382.0 382.3 382.4 382.7 383.1 ...
    CO2_FFF  (time, lev) float64 23.3 23.54 24.15 24.55 24.59 24.94 25.51 ...
    PSL      (time) float64 1.01e+05 1.01e+05 1.01e+05 1.01e+05 1.01e+05 ...
    Q        (time, lev) float64 3.539e-06 3.536e-06 3.53e-06 3.528e-06 ...
    U        (time, lev) float64 -26.6 -25.92 -24.6 -22.74 -21.19 -19.61 ...
    T        (time, lev) float64 270.0 266.9 264.1 259.1 253.8 249.6 246.6 ...
    CO2_OCN  (time, lev) float64 -2.257 -2.306 -2.439 -2.527 -2.535 -2.611 ...
    V        (time, lev) float6

### interpolate within the column to get point of measurement

In [6]:
%%time
rf_model = []
for i,rf in enumerate(RF):
    var = [model_colums[i][v] for v in varlist]
    rf_model.append(cam.interp_within_column(rf.PSXC,model_colums[i].Pm,*var,
                                             model_z_surface=model_colums[i].PS))
print rf_model[0]

<xarray.Dataset>
Dimensions:  (time: 2723)
Coordinates:
  * time     (time) datetime64[ns] 2016-01-15T12:57:26.000000043 ...
Data variables:
    O2_OCN   (time) float64 94.56 94.57 94.63 94.67 94.67 94.48 93.07 91.87 ...
    PS       (time) float64 990.8 990.8 990.8 990.8 990.8 990.8 990.8 990.8 ...
    PBLH     (time) float64 687.3 685.6 683.3 680.7 678.2 676.2 674.8 674.3 ...
    CO2      (time) float64 395.2 395.2 395.1 395.0 394.9 395.0 395.1 395.3 ...
    CO2_FFF  (time) float64 44.99 44.99 44.98 44.98 44.97 44.97 44.98 44.99 ...
    PSL      (time) float64 1.01e+05 1.01e+05 1.01e+05 1.01e+05 1.01e+05 ...
    Q        (time) float64 0.004186 0.004186 0.004164 0.004148 0.004123 ...
    U        (time) float64 3.447 3.559 4.412 4.849 5.254 5.55 5.942 6.167 ...
    T        (time) float64 282.1 281.9 280.9 279.9 278.5 277.0 276.1 275.3 ...
    CO2_OCN  (time) float64 -10.08 -10.09 -10.09 -10.09 -10.09 -10.08 -9.942 ...
    V        (time) float64 3.057 3.196 4.248 4.743 5.109 5.308 5

In [8]:
dir_out = os.path.join(os.path.dirname(gv.flight_file[0]),'..','cesm_flight_data')
if not os.path.exists(dir_out):
    call(['mkdir','-p',dir_out])
    
for i,rf in enumerate(rf_model):
    file_out = '.'.join([case,os.path.basename(gv.flight_file[i])])
    file_out = os.path.join(dir_out,file_out)
    print(file_out)
    rf.to_netcdf(file_out,unlimited_dims='time')

/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf01.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf02.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf03.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf04.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf05.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf06.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../cesm_flight_data/bgeos5.B20TRC5CN.f09_g16.BPRD_orcas_sci.004.ORCASrf07.merge10.20170526.nc
/glade/p/work/mclong/orcas/gv_merged_data/../ces

In [10]:
#fig = plt.figure(figsize=(15,15))
#ax = fig.add_subplot(111)
#p.O2_OCN.T.plot(ax=ax,vmin=60,vmax=140)
#ax.scatter(rf.time.values,rf.Pm.values,c=rf.O2_OCN.values,s=100,vmin=60,vmax=140,
#           edgecolors='k')
#ax.set_ylim([1013,200])
RF[0]

<xarray.Dataset>
Dimensions:         (time: 2723)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-15T12:57:26.000000043 ...
Data variables:
    Year            (time) float64 ...
    flt             (time) float64 ...
    DOY             (time) timedelta64[ns] ...
    UTC             (time) timedelta64[ns] ...
    ALTG_SRTM       (time) float64 ...
    ATTACK          (time) float64 ...
    ATX             (time) float64 ...
    BLATA           (time) float64 ...
    BLONGA          (time) float64 ...
    BNORMA          (time) float64 ...
    CONC2DCA_LWO    (time) float64 ...
    CONCD_LWI       (time) float64 ...
    CONCN           (time) float64 ...
    CONCU_RWO       (time) float64 ...
    CONCU100_RWO    (time) float64 ...
    CONCU500_RWO    (time) float64 ...
    DBAR2DCA_LWO    (time) float64 ...
    DBARD_LWI       (time) float64 ...
    DBARU_RWO       (time) float64 ...
    DPXC            (time) float64 ...
    EWX             (time) float64 ...
    GGALT 